In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import scipy as scipy

In [3]:
import scipy.fftpack
from scipy import signal
from time import gmtime, strftime
from scipy.signal import butter, lfilter

In [14]:
from sklearn.decomposition import PCA

In [4]:
def bandpass_filter(data, lowcut, highcut, signal_freq, filter_order):

        """

        Method responsible for creating and applying Butterworth filter.

        :param deque data: raw data

        :param float lowcut: filter lowcut frequency value

        :param float highcut: filter highcut frequency value

        :param int signal_freq: signal frequency in samples per second (Hz)

        :param int filter_order: filter order

        :return array: filtered data

        """

        nyquist_freq = 0.5 * signal_freq

        low = lowcut / nyquist_freq

        high = highcut / nyquist_freq

        b, a = butter(filter_order, [low, high], btype="band")

        y = lfilter(b, a, data)

        return y

In [5]:
data_info = pd.read_csv('D:/Parkinson Data L-Dopa Test/data_info.csv')

In [8]:
tremor_data = data_info[data_info['dyskinesiaScore']=='Score']

In [11]:
dat2 = tremor_data.reset_index()

In [12]:
distance = []
dis_variance = []
dis_median = []
dis_80 = []
dis_95 = []
for i in tqdm.tqdm(range(len(dat2))):
    x = pd.read_csv(dat2['Path'][i],sep='\t').astype(float)
    x.columns = ['timestamp','X','Y','Z','W']
    x['timestamp'] = (x['timestamp']-x['timestamp'][0])
    if(x['X'].isnull().sum()>0):
        distance.append(np.nan)
        dis_variance.append(np.nan)
        dis_median.append(np.nan)
        dis_80.append(np.nan)
        dis_95.append(np.nan)
    else:
        x1 = bandpass_filter(x['X'], lowcut=3,highcut=8, signal_freq=50,filter_order=2)
        y1 = bandpass_filter(x['Y'], lowcut=3,highcut=8, signal_freq=50,filter_order=2)
        z1 = bandpass_filter(x['Z'], lowcut=3,highcut=8, signal_freq=50,filter_order=2)
        v = np.sqrt((x1.cumsum()*0.02)*(x1.cumsum()*0.02)+(y1.cumsum()*0.02)*(y1.cumsum()*0.02)+(z1.cumsum()*0.02)*(z1.cumsum()*0.02))
        s = v*0.02
        total_distance = np.sum(s)    
        distance.append(total_distance/(len(x)/50))
        dis_variance.append(np.var(v))
        dis_median.append(np.median(v))
        dis_80.append(np.percentile(v,80))
        dis_95.append(np.percentile(v,95))

dat2['distance'] = distance
dat2['dis_var'] = dis_variance
dat2['dis_median'] = dis_median
dat2['dis_80'] = dis_80
dat2['dis_95'] = dis_95

100%|██████████| 660/660 [00:10<00:00, 64.24it/s]


In [15]:
pca_abs_mean = []
pca_mean = []
pca_var = []
pca_abs_max = []
exp_var = []
c_mean = []
c_median = []
c_std = []
#dat2 = tremor_data[(tremor_data['task'].str.contains('ftn')&(tremor_data['site']=='Boston'))].reset_index()
for i in tqdm.tqdm(range(len(dat2))):
    x = pd.read_csv(dat2['Path'][i],sep='\t').astype(float)
    x.columns = ['timestamp','X','Y','Z','W']
    x['timestamp'] = (x['timestamp']-x['timestamp'][0])
    if(x['X'].isnull().sum()>0):
        pca_abs_mean.append(np.nan)
        pca_mean.append(np.nan)
        pca_var.append(np.nan)
        pca_abs_max.append(np.nan)
        exp_var.append(np.nan)
        c_mean.append(disc_mean)
        c_median.append(disc_median)
        c_std.append(disc_std)
    else:
        x1 = bandpass_filter(x['X'], lowcut=3,highcut=8, signal_freq=50,filter_order=2)
        y1 = bandpass_filter(x['Y'], lowcut=3,highcut=8, signal_freq=50,filter_order=2)
        z1 = bandpass_filter(x['Z'], lowcut=3,highcut=8, signal_freq=50,filter_order=2)
        px = x1.cumsum().cumsum()*0.02*0.02
        py = y1.cumsum().cumsum()*0.02*0.02
        pz = z1.cumsum().cumsum()*0.02*0.02
        p = pd.DataFrame([px,py,pz]).transpose()
        pca = PCA().fit(p)
        exp_var.append(pca.explained_variance_ratio_[2])
        pca = PCA(n_components=3)
        pca.fit(p)
        x_pca = pca.transform(p)
        pca_abs_mean.append(np.mean(np.abs(x_pca[:,2])))
        pca_mean.append(np.mean(x_pca[:,2]))
        pca_var.append(np.var(x_pca[:,2]))
        pca_abs_max.append(np.max(np.abs(x_pca[:,2])))
        
        xc = np.median(px)
        yc = np.median(py)
        zc = np.median(pz)
        dis_c = np.sqrt((x1-xc)*(x1-xc)+(y1-yc)*(y1-yc)+(z1-zc)*(z1-zc))
        disc_mean = np.mean(dis_c)
        disc_median = np.median(dis_c)
        disc_std = np.std(dis_c)
        
        c_mean.append(disc_mean)
        c_median.append(disc_median)
        c_std.append(disc_std)
dat2['pca_abs_mean'] = pca_abs_mean
dat2['pca_mean'] = pca_mean
dat2['pca_var'] = pca_var
dat2['pca_abs_max'] = pca_abs_max
dat2['exp_var'] = exp_var
dat2['c_mean'] = c_mean
dat2['c_median'] = c_median
dat2['c_std'] = c_std

100%|██████████| 660/660 [00:16<00:00, 39.24it/s]


In [17]:
powerx = []
powery = []
powerz = []
f_s = 50
for i in tqdm.tqdm(range(len(dat2))):
    x = pd.read_csv(dat2['Path'][i],sep='\t').astype(float)
    x.columns= ['time','X','Y','Z','W']
    data1 = x['X']
    data2 = x['Y']
    data3 = x['Z']
    freqs = scipy.fftpack.fftfreq(len(x['X'])) * f_s
    if(x['X'].isnull().sum()>0):
        powerx.append(np.nan)
        powery.append(np.nan)
        powerz.append(np.nan)
    else:    
        p = pd.DataFrame([data1,data2,data3]).transpose()
        pca = PCA(n_components=3)
        pca.fit(p)
        x_pca = pca.transform(p)
        x1 = np.abs(scipy.fftpack.fft(x_pca[:,0]))[(freqs>=1)&(freqs<=8)].sum()
        y1 = np.abs(scipy.fftpack.fft(x_pca[:,1]))[(freqs>=1)&(freqs<=8)].sum()
        z1 = np.abs(scipy.fftpack.fft(x_pca[:,2]))[(freqs>=1)&(freqs<=8)].sum()
        powerx.append(x1)
        powery.append(y1)
        powerz.append(z1)
dat2['powerx'] = powerx
dat2['powery'] = powery
dat2['powerz'] = powerz

100%|██████████| 660/660 [00:07<00:00, 90.77it/s] 


In [19]:
angle_median = []
angle_mean = []
angle_std = []
for i in tqdm.tqdm(range(len(dat2))):
    x = pd.read_csv(dat2['Path'][i],sep='\t').astype(float)
    x.columns= ['time','X','Y','Z','W']
    if(x['X'].isnull().sum()>0):
        angle_median.append(np.nan)
        angle_mean.append(np.nan)
        angle_std.append(np.nan)
    else:    
        x1 = bandpass_filter(x['X'], lowcut=2,highcut=20, signal_freq=50,filter_order=2)
        y1 = bandpass_filter(x['Y'], lowcut=2,highcut=20, signal_freq=50,filter_order=2)
        z1 = bandpass_filter(x['Z'], lowcut=2,highcut=20, signal_freq=50,filter_order=2)
        new = pd.DataFrame([x1,y1,z1]).transpose()
        new2 = new[0:-1].values
        new3 = new[1:].values
        new2sum = np.sqrt(new2[:,0]*new2[:,0]+new2[:,1]*new2[:,1]+new2[:,2]*new2[:,2])
        new3sum = np.sqrt(new3[:,0]*new3[:,0]+new3[:,1]*new3[:,1]+new3[:,2]*new3[:,2])
        newsum = new2*new3
        cos = pd.DataFrame(newsum).sum(axis=1)/(new2sum*new3sum)
        arccos = np.arccos(cos)/3.1415926*180
        angle_median.append(np.median(arccos))
        angle_mean.append(np.mean(arccos))
        angle_std.append(np.std(arccos))

dat2['angle_median'] = angle_median
dat2['angle_mean'] = angle_mean
dat2['angle_std'] = angle_std

 50%|█████     | 333/660 [00:08<00:07, 40.90it/s]C:\Users\wangs\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in arccos
C:\Users\wangs\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3858: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)
100%|██████████| 660/660 [00:14<00:00, 44.08it/s]


In [20]:
def SampEn(U, m, r):
    std = np.std(U)
    def _maxdist(x_i, x_j):
        return max([abs(ua - va) for ua, va in zip(x_i, x_j)])

    def _phi(m):
        x = [[U[j] for j in range(i, i + m - 1 + 1)] for i in range(N - m + 1)]
        C = [(len([1 for x_j in x if _maxdist(x_i, x_j) <= r*std])) - 1 for x_i in x]
        return sum(C)+0.0000001

    N = len(U)

    return np.log(_phi(m)/_phi(m+1))

In [21]:
def MSE(U,n,m,r): ## return the SampEn for U up to scale n
    mse = []
    for i in range(n):
        d = U[0:-1:(i+1)].values
        mse.append(SampEn(d,m,r))
    return np.array(mse)
        

In [25]:
mse_scorex = []
mse_scorey = []
mse_scorez = []

for i in tqdm.tqdm(range(len(dat2))):
    p = dat2['Path'].values[i]
    test = pd.read_csv(p,sep='\t').astype(float)
    test.columns = ['time','X','Y','Z','M']
    test['time'] = test['time']-test['time'].values[0]
    mse_scorex.append(MSE(test['X'],10,2,0.2))
    mse_scorey.append(MSE(test['Y'],10,2,0.2))
    mse_scorez.append(MSE(test['Z'],10,2,0.2))


100%|██████████| 660/660 [1:32:08<00:00,  8.95s/it]


In [26]:
score = pd.concat([pd.DataFrame(mse_scorex),pd.DataFrame(mse_scorey),pd.DataFrame(mse_scorez)],axis=1)

In [27]:
score.columns = ['x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',
                'y1','y2','y3','y4','y5','y6','y7','y8','y9','y10',
                'z1','z2','z3','z4','z5','z6','z7','z8','z9','z10']

In [28]:
data = pd.concat([dat2,score],axis=1)

In [33]:
dat3 = data.copy().drop(['tremorScore','bradykinesiaScore','index','idx','Path'],axis=1)
dat3.to_csv('D:/Parkinson Data L-Dopa Test/dyskinesia_test.csv',index=False)